# 导入数据

In [ ]:
"""author Binny"""
# 忽略警告提示
import warnings
warnings.filterwarnings('ignore')
import sys
# 导入处理数据包
import pandas as pd
import numpy as np

train_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\train.csv')
test_data = pd.read_csv(r'D:\pythonProject1\ML_for_kaggle\data\titanic\test.csv')
full = train_data.append(test_data, ignore_index=True)
print('训练集大小：', train_data.shape, '测试集大小：', test_data.shape)
print('合并后的数据集大小：', full.shape)
print('合并后的数据集前5行：\n', full.head())

In [ ]:
'''
describe只能查看数据类型的描述统计信息，对于其他类型的数据不显示，比如字符串类型姓名（name），客舱号（Cabin）
这很好理解，因为描述统计指标是计算数值，所以需要该列的数据类型是数据。
Pandas Series.mean()函数返回给定Series对象中基础数据的平均值。
'''
# 获取数据类型列的描述统计信息
full.describe()

In [ ]:
full.info()

我们发现数据一共有1309行,其中Survided只有891行,说明有418行数据没有标签,也就是我们要预测的数据
Age(年龄)有1046行数据,说明有1309-1046=263行数据没有年龄,也就是说有263个乘客的年龄没有记录
Fare(船票价格)有1308行数据,只有一个乘客没有记录
Embarked(登船港口)有1307行数据,有两个乘客没有登船港口记录
Cabin(客舱号)只有295行数据,有1007个乘客没有客舱号记录


In [ ]:
def PrintMissing(your_dataframe): 
    """
    打印缺失数据统计信息
    :param 表示数据集
    :return: 返回一个新的dataframe，包含三列：'缺失数据'、'缺失率'、'数据类型'
    """
    drop_sum = your_dataframe.isnull().sum() # 统计每列缺失值个数
    drop_sum_not_zero = drop_sum[drop_sum > 0] # 只保留有缺失值的列
    df1 = pd.DataFrame({'缺失数据': drop_sum_not_zero}) # 创建一个新的dataframe，列名为'缺失数据'
    df2 = pd.DataFrame({'缺失率': drop_sum_not_zero/your_dataframe.shape[0]}) # 创建一个新的dataframe，列名为'缺失率'
    keys = drop_sum_not_zero.to_dict() # 将drop_sum_not_zero转换为字典
    df_dtypes = pd.DataFrame({'数据类型': {key: your_dataframe[key].dtype for key in keys}}) # 创建一个新的dataframe，列名为'数据类型'
    new_df = pd.concat([df1, df2.round(4), df_dtypes], axis=1)
    return new_df

print()
print('缺失数据统计如下（总数 %d）：' % full.shape[0])
PrintMissing(full)

# 数据清洗
很多机器学习算法为了训练模型，要求所传入的特征中不能有空值。

- 如果是数值类型，用平均值取代
- 如果是分类数据，用最常见的类别取代
- 使用模型预测缺失值，例如：K-NN

In [ ]:
# 年龄(Age)
full['Age'] = full['Age'].fillna(full['Age'].mean())
# 船票价格(Fare)
full['Fare'] = full['Fare'].fillna(full['Fare'].mean())
print()
print('数值字段处理后：')
PrintMissing(full)

In [ ]:
full.head()

In [ ]:
"""让我们看看登船港口的数据长什么样子"""
print('Embarked',list(pd.unique(full['Embarked'])))

出发地点：S=英国南安普顿Southampton
途径地点1：C=法国 瑟堡市Cherbourg
途径地点2：Q=爱尔兰 昆士敦Queenstown
可以看出S港口的人数最多，占了72.4%，所以用S港口填充缺失值

In [ ]:
'''
分类变量Embarked，看下最常见的类别，用其填充
'''
full['Embarked'].value_counts()

In [ ]:
full['Embarked'] = full['Embarked'].fillna('S')
#船舱号（Cabin）：查看里面数据长啥样
full["Cabin"].head()

In [ ]:
#缺失数据比较多，船舱号（Cabin）缺失值填充为U，表示未知（Uknow） 
full["Cabin"] = full["Cabin"].fillna( 'U' )

#检查数据处理是否正常
full.head()

In [ ]:
#查看最终缺失值处理情况，记住生存情况（Survived）这里一列是我们的标签，用来做机器学习预测的，不需要处理这一列
PrintMissing(full)

# 特征提取
'''
1.数值类型：
    乘客编号（PassengerId），
    年龄（Age），
    船票价格（Fare），
    同代直系亲属人数（SibSp），
    不同代直系亲属人数（Parch）
2.时间序列：无
3.分类数据：
    1）有直接类别的
        乘客性别（Sex）：男性（male），女性（female）
        登船港口（Embarked）：
            出发地点：S=英国.南安普顿（Southampton），
            途径地点1：C=法国.瑟堡市（Cherbourg），
            出发地点2：Q=爱尔兰.昆士敦（Queenstown）
        客舱等级（Pclass）：1=1等舱，2=2等舱，3=3等舱
    2）字符串类型：可能从这里面提取出特征来，也归到分类数据中
        乘客姓名（Name）
        船舱号（Cabin）
        船票编号（Ticket）
'''

In [ ]:
#查看性别数据这一列
full["Sex"].head()

In [ ]:
"""
将性别的值映射为数值
male为1,female为0
"""
# 如果为male就改为1，如果不是就改为0
sex_mapDict = {u'male': 1, u'female': 0}
full['Sex'] = full['Sex'].map(sex_mapDict)
full.head()

In [ ]:
#存放提取后的特征
embarkedDf = pd.DataFrame()

'''
使用get_dummies进行one-hot编码，产生虚拟变量（dummy variables），列名前缀是Embarked
'''
embarkedDf = pd.get_dummies( full["Embarked"] , prefix="Embarked" )
embarkedDf.head()

In [ ]:
# 添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full, embarkedDf], axis=1)

'''
因为已经使用登船港口(Embarked)进行了one-hot编码产生了它的虚拟变量（dummy variables）
所以这里把登船港口(Embarked)删掉
'''
full.drop("Embarked", axis=1, inplace=True)
full.head()

# 上面drop删除某一列代码解释：
# 因为drop(name,axis=1)里面指定了name是哪一列，比如指定的是A这一列，axis=1表示按行操作。
# 那么结合起来就是把A列里面每一行删除，最终结果是删除了A这一列.
# 简单来说，使用drop删除某几列的方法记住这个语法就可以了：drop([列名1,列名2],axis=1)

In [ ]:
'''
客舱等级(Pclass):
1=1等舱，2=2等舱，3=3等舱
'''
# 存放提取后的特征
pclassDf = pd.DataFrame()

# 使用get_dummies进行one-hot编码，列名前缀是Pclass
pclassDf = pd.get_dummies(full["Pclass"], prefix="Pclass")
pclassDf.head()

In [ ]:
# 添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full, pclassDf], axis=1)

# 删掉客舱等级（Pclass）这一列
full.drop("Pclass", axis=1, inplace=True)
full.head()

In [ ]:
'''
查看姓名这一列长啥样
注意到在乘客名字（Name）中，有一个非常显著的特点：
乘客头衔每个名字当中都包含了具体的称谓或者说是头衔，将这部分信息提取出来后可以作为非常有用一个新变量，可以帮助我们进行预测。
例如：
    Braund, Mr. Owen Harris
    Heikkinen, Miss. Laina
    Oliva y Ocana, Dona. Fermina
    Peter, Master. Michael J
'''
full["Name"].head()

In [ ]:
# 练习从字符串中提取头衔，例如Mr
# split用于字符串分割，返回一个列表
# 我们看到姓名中'Braund, Mr. Owen Harris'，逗号前面的是“名”，逗号后面是‘头衔. 姓’
name1 = 'Braund, Mr. Owen Harris'
'''
split用于字符串按分隔符分割，返回一个列表。这里按逗号分隔字符串
也就是字符串'Braund, Mr. Owen Harris'被按分隔符,'拆分成两部分[Braund,Mr. Owen Harris]
你可以把返回的列表打印出来瞧瞧，这里获取到列表中元素序号为1的元素，也就是获取到头衔所在的那部分，即Mr. Owen Harris这部分
'''
# Mr. Owen Harris
str1 = name1.split(',')[1]
'''
继续对字符串Mr. Owen Harris按分隔符'.'拆分，得到这样一个列表[Mr, Owen Harris]
这里获取到列表中元素序号为0的元素，也就是获取到头衔所在的那部分Mr
'''
# Mr.
str2 = str1.split('.')[0]
# strip() 方法用于移除字符串头尾指定的字符（默认为空格）
str3 = str2.strip()
print(str3)

In [ ]:
'''
定义函数：从姓名中获取头衔
'''
def getTitle(name):
    str1 = name.split(',')[1]  # Mr. Owen Harris
    str2 = str1.split('.')[0]  # Mr
    # strip() 方法用于移除字符串头尾指定的字符（默认为空格）
    str3 = str2.strip()
    return str3
# 存放提取后的特征
titleDf = pd.DataFrame()
# map函数：对Series每个数据应用自定义的函数计算
titleDf["Name"] = full["Name"].map(getTitle)
titleDf

In [ ]:
'''
定义以下几种头衔类别：
    Officer：政府官员
    Royalty：王室（皇室）
    Mr：已婚男士
    Mrs：已婚妇女
    Miss：年轻未婚女子
    Master：有技能的人/教师
'''
# 姓名中头衔字符串与定义头衔类别的映射关系
title_mapDict = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir":        "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess": "Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr":         "Mr",
    "Mrs":        "Mrs",
    "Miss":       "Miss",
    "Master":     "Master",
    "Lady":       "Royalty"
}
# map函数：对Series每个数据应用自定义的函数计算
titleDf["Title"] = titleDf["Name"].map(title_mapDict)
titleDf.head()
# 删除Name这一列
titleDf.drop("Name", axis=1, inplace=True)
print("Title", list(pd.unique(titleDf["Title"])))
# 使用get_dummies进行one-hot编码
titleDf = pd.get_dummies(titleDf["Title"])
titleDf.head()

In [ ]:
# 添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full, titleDf], axis=1)

# 删掉姓名这一列
full.drop("Name", axis=1, inplace=True)
full.head()

In [ ]:
# 存放客舱号信息
cabinDf = pd.DataFrame()

'''
客场号的类别值是首字母，例如：
C85 类别映射为首字母C
'''
full['Cabin'] = full['Cabin'].map(lambda c: c[0])

# 使用get_dummies进行one-hot编码，列名前缀是Cabin
cabinDf = pd.get_dummies(full['Cabin'], prefix='Cabin')

cabinDf.head()

In [ ]:
# 添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full, cabinDf], axis=1)

# 删掉客舱号这一列
full.drop("Cabin", axis=1, inplace=True)
full.head()

In [ ]:
# 存放家庭信息
familyDf = pd.DataFrame()

'''
家庭人数=同代直系亲属数（Parch）+不同代直系亲属数（SibSp）+乘客自己
（因为乘客自己也是家庭成员的一个，所以这里加1）
'''
familyDf["FamilySize"] = full["Parch"] + full["SibSp"] + 1

'''
家庭类别：
    小家庭（Family_Single）：家庭人数=1
    中等家庭（Family_Small）: 2<=家庭人数<=4
    大家庭（Family_Large）: 家庭人数>=5
'''
# if 条件为真的时候返回if前面内容，否则返回0
familyDf['Family_Single'] = familyDf["FamilySize"].map(
    lambda s: 1 if s == 1 else 0)
familyDf['Family_Small'] = familyDf["FamilySize"].map(
    lambda s: 1 if 2 <= s <= 4 else 0)
familyDf['Family_Large'] = familyDf["FamilySize"].map(
    lambda s: 1 if 5 <= s else 0)

familyDf.head()

In [ ]:
# 添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克号数据集full
full = pd.concat([full, familyDf], axis=1)
full.head()

In [ ]:
#相关性矩阵
corrDf = full.corr() 
corrDf.head()

In [ ]:
'''
查看各个特征与生成情况（Survived）的相关系数，
ascending=False表示按降序排列
'''
corrDf["Survived"].sort_values(ascending=False)

In [ ]:
# 特征选择
full_X = pd.concat([titleDf,  # 头衔
                    pclassDf,  # 客舱等级
                    familyDf,  # 家庭大小
                    full["Fare"],  # 船票价格
                    cabinDf,  # 船舱号
                    embarkedDf,  # 登船港口
                    full["Sex"]  # 性别
                    ], axis=1)
full_X.head()

In [ ]:
'''
1）坦尼克号测试数据集因为是我们最后要提交给Kaggle的，里面没有生存情况的值，所以不能用于评估模型。
    我们将Kaggle泰坦尼克号项目给我们的测试数据，叫做预测数据集（记为pred,也就是预测英文单词predict的缩写）。
    也就是我们使用机器学习模型来对其生存情况就那些预测。
2）我们使用Kaggle泰坦尼克号项目给的训练数据集，做为我们的原始数据集（记为source），
    从这个原始数据集中拆分出训练数据集（记为train：用于模型训练）和测试数据集（记为test：用于模型评估）。

'''

'''
TRAIN_ROWS 是我们在最开始合并数据前知道的，原始数据集有总共有 TRAIN_ROWS 条数据
从特征集合full_X中提取原始数据集提取前 TRAIN_ROWS 行数据时，我们要减去1，因为行号是从0开始的。
'''
TRAIN_ROWS = train_data.shape[0] # 训练数据集有多少行
# 原始数据集：特征
source_X = full_X.loc[0:TRAIN_ROWS-1, :]
# 原始数据集：标签
source_y = full.loc[0:TRAIN_ROWS-1, 'Survived']
# 预测数据集：特征
pred_X = full_X.loc[TRAIN_ROWS:, :]

# 上面代码解释：
# TRAIN_ROWS 行前面的数据是测试数据集，TRAIN_ROWS 行之后的数据是预测数据集。[TRAIN_ROWS:,:]就是
# 从 TRAIN_ROWS 行开始到最后一行作为预测数据集

In [ ]:
'''
从原始数据集（source）中拆分出训练数据集（用于模型训练 train），测试数据集（用于模型评估 test）
train_test_split 是交叉验证中常用的函数，功能是从样本中随机的按比例选取 train data 和 test data
train_data：所要划分的样本特征集
train_target：所要划分的样本结果
test_size：样本占比，如果是整数的话就是样本的数量
'''
from sklearn.model_selection import train_test_split

# 建立模型用的训练数据集和测试数据集
train_X, test_X, train_y, test_y = train_test_split(source_X,
                                                    source_y,
                                                    train_size=.8)

# 输出数据集大小
print('原始数据集特征：', source_X.shape,
      '训练数据集特征：', train_X.shape,
      '测试数据集特征：', test_X.shape)

print('原始数据集标签：', source_y.shape,
      '训练数据集标签：', train_y.shape,
      '测试数据集标签：', test_y.shape)

In [ ]:
# 原始数据查看
source_y.head()

In [ ]:
train_X.head()

In [ ]:
train_y.head()

In [ ]:
# 第1步：导入算法
from sklearn.linear_model import LogisticRegression
# 第2步：创建模型：逻辑回归（logisic regression）
model = LogisticRegression(solver='liblinear', class_weight="balanced")
# 第3步：训练模型
model.fit(train_X, train_y)
# 分类问题，score得到的是模型的正确率
model.score(test_X , test_y )

In [ ]:
# 使用机器学习模型，对预测数据集中的生存情况进行预测
pred_Y = model.predict(pred_X)

'''
生成的预测值是浮点数（0.0,1,0）
但是Kaggle要求提交的结果是整型（0,1）
所以要对数据类型进行转换
'''
pred_Y = pred_Y.astype(int)
# 乘客id
passenger_id = full.loc[TRAIN_ROWS:, 'PassengerId']
# 数据框：乘客id，预测生存情况的值
predDf = pd.DataFrame(
    {"passenger_id": passenger_id,
     "Survived": pred_Y})
predDf.shape
predDf.head()
# 保存结果
predDf.to_csv('titanic_pred.csv', index=False)

In [90]:
# 将source_X和source_y进行拼接，得到原始数据集
source = pd.concat([source_X, source_y], axis=1)
# 将source分成训练数据集和测试数据集
train_cathy, test_cathy = train_test_split(source, train_size=.8)

In [91]:
from autogluon.tabular import TabularDataset, TabularPredictor
label = 'Survived'
save_path = 'agModels-predictClass'  # specifies folder to store trained models
predictor = TabularPredictor(label=label, path=save_path,).fit(train_cathy)



Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.8.2
Python Version:     3.10.4
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.18363
Disk Space Avail:   30.39 GB / 170.16 GB (17.9%)
Train Data Rows:    712
Train Data Columns: 27
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0.0, 1.0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    19266.72 MB
	Train Data (Original)  Memory Usage: 0.05 MB (0.0% of available memory)
	Inferring data

In [93]:
y_pred = predictor.predict(test_cathy.drop(columns=[label]))
print("用测试集中的数据进行预测的情况为：\n", y_pred)

用测试集中的数据进行预测的情况为：
 495    0.0
648    0.0
278    0.0
31     1.0
255    1.0
      ... 
780    1.0
837    0.0
215    1.0
833    0.0
372    0.0
Name: Survived, Length: 179, dtype: float64


In [94]:
predictor.leaderboard(test_cathy, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.837989,0.790210,0.081233,0.073580,0.556679,0.081233,0.073580,0.556679,1,True,6
1,RandomForestGini,0.832402,0.790210,0.096945,0.061965,0.576512,0.096945,0.061965,0.576512,1,True,5
2,NeuralNetTorch,0.826816,0.881119,0.013992,0.007995,2.794698,0.013992,0.007995,2.794698,1,True,12
3,WeightedEnsemble_L2,0.826816,0.881119,0.016990,0.008995,3.583967,0.002999,0.001000,0.789269,2,True,14
4,NeuralNetFastAI,0.821229,0.874126,0.108938,0.010995,0.712219,0.108938,0.010995,0.712219,1,True,10
5,LightGBMXT,0.815642,0.874126,0.003995,0.003015,0.458261,0.003995,0.003015,0.458261,1,True,3
6,CatBoost,0.815642,0.867133,0.007996,0.003999,0.786182,0.007996,0.003999,0.786182,1,True,7
7,ExtraTreesGini,0.815642,0.776224,0.086776,0.063978,0.553760,0.086776,0.063978,0.553760,1,True,8
8,LightGBM,0.810056,0.874126,0.003997,0.002998,0.623580,0.003997,0.002998,0.623580,1,True,4
9,LightGBMLarge,0.810056,0.860140,0.003998,0.002999,0.574417,0.003998,0.002999,0.574417,1,True,13


In [96]:
predictor.evaluate(test_cathy, silent=True)

{'accuracy': 0.8268156424581006,
 'balanced_accuracy': 0.7969696969696969,
 'mcc': 0.6298122927747527,
 'roc_auc': 0.8732542819499342,
 'f1': 0.7479674796747967,
 'precision': 0.8518518518518519,
 'recall': 0.6666666666666666}

In [106]:
y_forecast = predictor.predict(pred_X)
# 将y_forecast转化为dataframe
y_forecast = pd.DataFrame(y_forecast)
# 将y_forecast添加新的一列PassengerId,要求放在第一列
y_forecast.insert(0, 'PassengerId', passenger_id)
y_forecast
# 保存为csv文件
y_forecast.to_csv('titanic_pred_autogluon.csv', index=False)

In [98]:
# 将y_forecast保存为csv文件
y_forecast.to_csv('titanic_pred_autogluon.csv', index=False)